In [1]:
import os
os.environ["OPENBLAS_NUM_THREADS"] = "1"  # For implicit ALS

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
# If the library is not installed - unlock the field 
!{sys.executable} -m pip install rectools

/bin/bash: {sys.executable}: command not found


In [4]:
pip install rectools

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.0/89.0 kB 528.8 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.0/89.0 kB 1.8 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.5/13.5 MB 22.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.5/188.5 kB 12.2 MB/s eta 0:00:00
  Attempting uninstall: pybind11
    Found existing installation: pybind11 2.10.0
    Uninstalling pybind11-2.10.0:
      Successfully uninstalled pybind11-2.10.0
  Attempting uninstall: Markdown
    Found existing installation: Markdown 3.3.7
    Uninstalling Markdown-3.3.7:
      Successfully uninstalled Markdown-3.3.7
Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install optuna

Note: you may need to restart the kernel to use updated packages.


In [6]:
import pandas as pd
import numpy as np
import typing as tp
import requests
import zipfile as zf
import optuna
import time

from rectools.metrics import Precision, Recall, MAP, calc_metrics
from rectools.models import PopularModel, RandomModel, ImplicitALSWrapperModel
from rectools import Columns
from rectools.dataset import Dataset
from rectools.models import ImplicitALSWrapperModel, LightFMWrapperModel

from tqdm import tqdm

from lightfm import LightFM
from implicit.bpr import BayesianPersonalizedRanking
from implicit.lmf import LogisticMatrixFactorization
from implicit.als import AlternatingLeastSquares

# LOAD DATA 

In [7]:
url = 'https://github.com/irsafilo/KION_DATASET/raw/f69775be31fa5779907cf0a92ddedb70037fb5ae/data_original.zip'

In [8]:
req = requests.get(url, stream=True)

with open('kion.zip', 'wb') as fd:
    total_size_in_bytes = int(req.headers.get('Content-Length', 0))
    progress_bar = tqdm(desc='kion dataset download', total=total_size_in_bytes, unit='iB', unit_scale=True)
    for chunk in req.iter_content(chunk_size=2 ** 20):
        progress_bar.update(len(chunk))
        fd.write(chunk)

kion dataset download:  79%|███████▊  | 61.9M/78.8M [00:00<00:00, 310MiB/s]

In [9]:
files = zf.ZipFile('kion.zip','r')
files.extractall()
files.close()

In [10]:
%%time
users = pd.read_csv('data_original/users.csv')
items = pd.read_csv('data_original/items.csv')
interactions = pd.read_csv('data_original/interactions.csv')

# interactions.rename(
#     columns={
#         'track_id': Columns.Item,
#         'last_watch_dt': Columns.Datetime,
#         'total_dur': Columns.Weight
#     },
#     inplace=True)

# interactions[Columns.Datetime] = pd.to_datetime(interactions[Columns.Datetime])

CPU times: user 3.46 s, sys: 744 ms, total: 4.2 s
Wall time: 4.21 s


# Preprocess

In [11]:
Columns.Datetime = 'last_watch_dt'

In [12]:
interactions.drop(interactions[interactions[Columns.Datetime].str.len() != 10].index, inplace=True)

In [13]:
interactions.head()

,user_id,item_id,last_watch_dt,total_dur,watched_pct
0,176549,9506,2021-05-11,4250,72.0
1,699317,1659,2021-05-29,8317,100.0
2,656683,7107,2021-05-09,10,0.0
3,864613,7638,2021-07-05,14483,100.0
4,964868,9506,2021-04-30,6725,100.0



<center><div class="alert alert-block alert-warning" style="margin: 2em; line-height: 1.7em; font-family: Verdana;">
    <b style="font-size: 18px;">Добавим трех пользователей, для проверки рекомендаций</b><br>
</div></center>


In [14]:
np.random.seed(42)

In [15]:
# Найдем id для 1ого юзера
interactions[interactions[Columns.User] == 100000] 

,user_id,item_id,last_watch_dt,total_dur,watched_pct


In [16]:
# Найдем id для 2ого юзера
interactions[interactions[Columns.User] == 200000] 

,user_id,item_id,last_watch_dt,total_dur,watched_pct


In [17]:
# Найдем id для 3его юзера
interactions[interactions[Columns.User] == 400000] 

,user_id,item_id,last_watch_dt,total_dur,watched_pct


In [18]:
def random_dates(start, end, n=10):

    ndays = (end - start).days + 1
    return pd.to_timedelta(np.random.randint(0, ndays, n), unit='D') + start

## 1й пользователь, который смотрит только новые фильмы, с 2015 года

In [19]:
item_year = items.query("release_year > 2015")[Columns.Item].unique()
new_users_last_year = pd.DataFrame()
new_users_last_year['item_id'] = item_year[np.random.randint(len(item_year), size=10)]
new_users_last_year[Columns.User] = 100000
new_users_last_year[Columns.Datetime] = random_dates(pd.to_datetime(interactions[Columns.Datetime].min()), 
                                           pd.to_datetime((interactions[Columns.Datetime].max())))
new_users_last_year['total_dur'] = np.random.randint(interactions['total_dur'].quantile(0.1), interactions['total_dur'].quantile(0.9), size=10)
new_users_last_year['watched_pct'] = np.random.randint(100, size=10)

In [20]:
new_users_last_year.head()

,item_id,user_id,last_watch_dt,total_dur,watched_pct
0,4759,100000,2021-05-26,10625,21
1,10754,100000,2021-06-08,13845,88
2,1164,100000,2021-07-07,6991,48
3,12492,100000,2021-06-20,2475,90
4,5073,100000,2021-06-24,5353,58


## 2й пользователь, который смотрит только фэнтези.

In [21]:
item_fi = items[Columns.Item][items['genres'].apply(lambda x: 'фэнтези' in x)].unique()
new_users_fi = pd.DataFrame()
new_users_fi['item_id'] = item_fi[np.random.randint(len(item_fi), size=10)]
new_users_fi[Columns.User] = 200000
new_users_fi[Columns.Datetime] = random_dates(pd.to_datetime(interactions[Columns.Datetime].min()), 
                                           pd.to_datetime((interactions[Columns.Datetime].max())))
new_users_fi['total_dur'] = np.random.randint(interactions['total_dur'].quantile(0.1), interactions['total_dur'].quantile(0.9), size=10)
new_users_fi['watched_pct'] = np.random.randint(100, size=10)

In [22]:
new_users_fi.head()

,item_id,user_id,last_watch_dt,total_dur,watched_pct
0,4711,200000,2021-07-21,8475,70
1,7582,200000,2021-05-02,10275,43
2,7704,200000,2021-07-25,11058,7
3,7582,200000,2021-04-02,7555,46
4,964,200000,2021-05-24,2654,34


## И 3й пользователь, который фанат фильмов США.

In [23]:
item_country = items.query("countries == 'США'")[Columns.Item].unique()
new_users_country = pd.DataFrame()
new_users_country['item_id'] = item_country[np.random.randint(len(item_country), size=10)]
new_users_country[Columns.User] = 400000
new_users_country[Columns.Datetime] = random_dates(pd.to_datetime(interactions[Columns.Datetime].min()), 
                                           pd.to_datetime((interactions[Columns.Datetime].max())))
new_users_country['total_dur'] = np.random.randint(interactions['total_dur'].quantile(0.1), interactions['total_dur'].quantile(0.9), size=10)
new_users_country['watched_pct'] = np.random.randint(100, size=10)

In [24]:
new_users_country.head()

,item_id,user_id,last_watch_dt,total_dur,watched_pct
0,7330,400000,2021-04-25,4929,44
1,11354,400000,2021-08-21,12227,64
2,716,400000,2021-03-26,14594,88
3,11209,400000,2021-06-15,4901,70
4,363,400000,2021-04-29,6373,8


## Добавим наших юзеров в наши данные

In [25]:
print(f'Размер данных до добавления юзеров {interactions.shape[0]}')

interactions = pd.concat([interactions, new_users_last_year, new_users_fi, new_users_country])

print(f'Размер данных после добавления юзеров {interactions.shape[0]}')

Размер данных до добавления юзеров 5476251
Размер данных после добавления юзеров 5476281


#### Все верно! Так как про каждого из юзеров у нас было создано по 10 записей, общий размер данных увеличился ровно на 30 записей


<center><div class="alert alert-block alert-warning" style="margin: 2em; line-height: 1.7em; font-family: Verdana;">
    <b style="font-size: 18px;"> Генерация наших матриц между юзером и итемов. item_features, user_fetures</b><br>
</div></center>

In [26]:
#  Переведем даты в тип datetime и найдем max_date
interactions[Columns.Datetime] = pd.to_datetime(interactions[Columns.Datetime], format='%Y-%m-%d')
max_date = interactions[Columns.Datetime].max()

# Создадим веса взаимодействий итемов и юзеров. 
## Сделаем свою разбаловку по % просмотра фильма
###  watched_pct => 90% = 10 
###   65 <= watched_pct < 90% = 7
### 30 <= watched_pct < 65% = 5
###  10 <= watched_pct < 30% = 3
### watched_pct < 10% = 1

In [27]:
def create_weight_from_watched_pct(x):
    if x >= 90:
        return 10
    elif x >= 65:
        return 7
    elif x >= 30:
        return 5
    elif x >= 10:
        return 3
    return 1

interactions[Columns.Weight] = interactions['watched_pct'].apply(create_weight_from_watched_pct)

interactions[Columns.Weight].value_counts()

10    1772275
1     1754939
3      907698
5      670896
7      370473
Name: weight, dtype: int64

In [28]:
# Разобьем все на тест, траин
train = interactions[interactions[Columns.Datetime] < max_date - pd.Timedelta(days=7)].copy()
test = interactions[interactions[Columns.Datetime] >= max_date - pd.Timedelta(days=7)].copy()

print(f"train: {train.shape}")
print(f"test: {test.shape}")

train: (4985298, 6)
test: (490983, 6)


In [29]:
# Удалим всех итемы с длиной меньше 331с
train.drop(train.query("total_dur < 331").index, inplace=True)

kion dataset download: 100%|██████████| 78.8M/78.8M [00:20<00:00, 310MiB/s]

In [30]:
# отфильтруем холодных пользователей из теста
cold_users = set(test[Columns.User]) - set(train[Columns.User])

In [31]:
# Удаляем их из теста
test.drop(test[test[Columns.User].isin(cold_users)].index, inplace=True)

In [32]:
train.query("user_id == 400000")

,user_id,item_id,last_watch_dt,total_dur,watched_pct,weight
0,400000,7330,2021-04-25,4929,44.0,5
3,400000,11209,2021-06-15,4901,70.0,7
4,400000,363,2021-04-29,6373,8.0,1
5,400000,8780,2021-03-27,8613,87.0,7
6,400000,4423,2021-04-21,8726,0.0,1
7,400000,3155,2021-06-02,7250,7.0,1
8,400000,8504,2021-07-01,5318,87.0,7
9,400000,1761,2021-05-04,2104,62.0,5


# Prepare features

## User features

In [33]:
users.head()

,user_id,age,income,sex,kids_flg
0,973171,age_25_34,income_60_90,М,1
1,962099,age_18_24,income_20_40,М,0
2,1047345,age_45_54,income_40_60,Ж,0
3,721985,age_45_54,income_20_40,Ж,0
4,704055,age_35_44,income_60_90,Ж,0


In [34]:
users.isnull().sum()

user_id         0
age         14095
income      14776
sex         13831
kids_flg        0
dtype: int64

In [35]:
users.fillna('Unknown', inplace=True)

In [36]:
users.nunique()

user_id     840197
age              7
income           7
sex              3
kids_flg         2
dtype: int64

In [37]:
# Оставим только юзеров, которые есть в train
users = users.loc[users[Columns.User].isin(train[Columns.User])].copy()

In [38]:
# Добавим еще одну фичу, которая есть в данных 'kids_flg'

user_features_frames = []
for feature in ["sex", "age", "income", 'kids_flg']:
    feature_frame = users.reindex(columns=[Columns.User, feature])
    feature_frame.columns = ["id", "value"]
    feature_frame["feature"] = feature
    user_features_frames.append(feature_frame)
user_features = pd.concat(user_features_frames)
user_features.head()

,id,value,feature
0,973171,М,sex
1,962099,М,sex
3,721985,Ж,sex
4,704055,Ж,sex
5,1037719,М,sex


In [39]:
user_features.query(f"id == 973171")

,id,value,feature
0,973171,М,sex
0,973171,age_25_34,age
0,973171,income_60_90,income
0,973171,1,kids_flg


# Item features

In [40]:
items.isnull().sum() 

item_id             0
content_type        0
title               0
title_orig       4745
release_year       98
genres              0
countries          37
for_kids        15397
age_rating          2
studios         14898
directors        1509
actors           2619
description         2
keywords          423
dtype: int64

In [41]:
items = items.loc[items[Columns.Item].isin(train[Columns.Item])].copy()

In [42]:
items.head()

,item_id,content_type,title,title_orig,release_year,genres,countries,for_kids,age_rating,studios,directors,actors,description,keywords
0,10711,film,Поговори с ней,Hable con ella,2002.0,"драмы, зарубежные, детективы, мелодрамы",Испания,NaN,16.0,NaN,Педро Альмодовар,"Адольфо Фернандес, Ана Фернандес, Дарио Гранди...",Мелодрама легендарного Педро Альмодовара «Пого...,"Поговори, ней, 2002, Испания, друзья, любовь, ..."
1,2508,film,Голые перцы,Search Party,2014.0,"зарубежные, приключения, комедии",США,NaN,16.0,NaN,Скот Армстронг,"Адам Палли, Брайан Хаски, Дж.Б. Смув, Джейсон ...",Уморительная современная комедия на популярную...,"Голые, перцы, 2014, США, друзья, свадьбы, прео..."
2,10716,film,Тактическая сила,Tactical Force,2011.0,"криминал, зарубежные, триллеры, боевики, комедии",Канада,NaN,16.0,NaN,Адам П. Калтраро,"Адриан Холмс, Даррен Шалави, Джерри Вассерман,...",Профессиональный рестлер Стив Остин («Все или ...,"Тактическая, сила, 2011, Канада, бандиты, ганг..."
3,7868,film,45 лет,45 Years,2015.0,"драмы, зарубежные, мелодрамы",Великобритания,NaN,16.0,NaN,Эндрю Хэй,"Александра Риддлстон-Барретт, Джеральдин Джейм...","Шарлотта Рэмплинг, Том Кортни, Джеральдин Джей...","45, лет, 2015, Великобритания, брак, жизнь, лю..."
4,16268,film,Все решает мгновение,NaN,1978.0,"драмы, спорт, советские, мелодрамы",СССР,NaN,12.0,Ленфильм,Виктор Садовский,"Александр Абдулов, Александр Демьяненко, Алекс...",Расчетливая чаровница из советского кинохита «...,"Все, решает, мгновение, 1978, СССР, сильные, ж..."


In [43]:
items.nunique()

item_id         13965
content_type        2
title           13402
title_orig       9699
release_year      103
genres           2549
countries         666
for_kids            2
age_rating          6
studios            38
directors        7400
actors          11802
description     13741
keywords        13529
dtype: int64

### Genre

In [44]:
# Explode genres to flatten table
items["genre"] = items["genres"].str.lower().str.replace(", ", ",", regex=False).str.split(",")
genre_feature = items[["item_id", "genre"]].explode("genre")
genre_feature.columns = ["id", "value"]
genre_feature["feature"] = "genre"
genre_feature.head()

,id,value,feature
0,10711,драмы,genre
0,10711,зарубежные,genre
0,10711,детективы,genre
0,10711,мелодрамы,genre
1,2508,зарубежные,genre


### Content

In [45]:
content_feature = items.reindex(columns=[Columns.Item, "content_type"])
content_feature.columns = ["id", "value"]
content_feature["feature"] = "content_type"

In [46]:
content_feature

,id,value,feature
0,10711,film,content_type
1,2508,film,content_type
2,10716,film,content_type
3,7868,film,content_type
4,16268,film,content_type
...,...,...,...
15958,6443,series,content_type
15959,2367,series,content_type
15960,10632,series,content_type
15961,4538,series,content_type


## Добавим из фичей страну фильма и год выпуска фильма

In [47]:
country_feature = items.reindex(columns=[Columns.Item, 'countries'])
country_feature.columns = ["id", "value"]
country_feature["feature"] = "country"
country_feature.head()

,id,value,feature
0,10711,Испания,country
1,2508,США,country
2,10716,Канада,country
3,7868,Великобритания,country
4,16268,СССР,country


In [48]:
print(f'Кол-во разных дат в фиче год выпуска: {items.release_year.nunique()}')

Кол-во разных дат в фиче год выпуска: 103


In [49]:
# так как дат слишком много разобьем фильмы на 3 категории. Фильмые после 2015 года, с 2000 до 2015, и фильмы, которые выпускались ранее
release_year = items.reindex(columns=[Columns.Item, 'release_year'])
release_year.columns = ["id", "value"]
release_year["feature"] = "year"



In [50]:
def year_to_category(year):
    if year >= 2015:
        return 'cat_>2015'
    elif year >= 2000:
        return 'cat_>2000'
    else:
        return 'cat_<2000'
        

In [51]:
release_year = items.reindex(columns=[Columns.Item, 'release_year'])
release_year.columns = ["id", "value"]
release_year["feature"] = "year"
release_year["value"] = release_year["value"].apply(year_to_category)
release_year.head()

,id,value,feature
0,10711,cat_>2000,year
1,2508,cat_>2000,year
2,10716,cat_>2000,year
3,7868,cat_>2015,year
4,16268,cat_<2000,year


In [52]:
item_features = pd.concat((genre_feature, 
                           content_feature,
                           release_year,
                           country_feature     
                          ))

In [53]:
item_features.sample(10)

,id,value,feature
959,5767,series,content_type
8744,9485,зарубежные,genre
11714,5465,film,content_type
15874,7744,США,country
14444,4062,"США, Россия",country
8727,1311,series,content_type
15540,3120,драмы,genre
4936,8276,cat_<2000,year
14772,5939,ужасы,genre
1342,8941,film,content_type


In [54]:
%%time
dataset = Dataset.construct(
    interactions_df=train,
    user_features_df=user_features,
    cat_user_features=["sex", "age", "income", 
                      "kids_flg"],
    item_features_df=item_features,
    cat_item_features=["genre", "content_type", "year", "country"],
)

CPU times: user 2.31 s, sys: 187 ms, total: 2.49 s
Wall time: 2.5 s


In [55]:
TEST_USERS = test[Columns.User].unique()

# Metrics

In [56]:
# Будем оценивать только по метрикам от @8 - @10 
metrics_name = {
    'Precision': Precision,
    'Recall': Recall,
    'MAP': MAP,
}

metrics = {}
for metric_name, metric in metrics_name.items():
    for k in range(5, 11):
        metrics[f'{metric_name}@{k}'] = metric(k=k)
        

In [57]:
metrics

{'Precision@5': Precision(k=5),
 'Precision@6': Precision(k=6),
 'Precision@7': Precision(k=7),
 'Precision@8': Precision(k=8),
 'Precision@9': Precision(k=9),
 'Precision@10': Precision(k=10),
 'Recall@5': Recall(k=5),
 'Recall@6': Recall(k=6),
 'Recall@7': Recall(k=7),
 'Recall@8': Recall(k=8),
 'Recall@9': Recall(k=9),
 'Recall@10': Recall(k=10),
 'MAP@5': MAP(k=5, divide_by_k=False),
 'MAP@6': MAP(k=6, divide_by_k=False),
 'MAP@7': MAP(k=7, divide_by_k=False),
 'MAP@8': MAP(k=8, divide_by_k=False),
 'MAP@9': MAP(k=9, divide_by_k=False),
 'MAP@10': MAP(k=10, divide_by_k=False)}

# Models

In [ ]:
# Возьмем модели и прогоним через библиотеку Optuna для поиска наилучшей модели.  

In [61]:
def objective(trial, numRounds=100, RANDOM_STATE=42, NUM_THREADS=16, K_RECOS=10):
    
    global  TEST_USERS
    st_time = time.time()
    #Прописываем возможные гиперпараметры
    param = {
    'no_components': trial.suggest_int("no_components", 4, 37),
    "loss":  trial.suggest_categorical("loss", ["bpr", "warp", "logistic"]),
    "learning_rate": trial.suggest_float("learning_rate", 0.05, 0.4),
    "item_alpha": trial.suggest_float("item_alpha", 0, 2e-01),
    "user_alpha": trial.suggest_float("user_alpha", 0, 2e-01),
    "is_fitting_features": trial.suggest_categorical("is_fitting_features", [True, False]),
    }
    
    # Кол-во эпох под LightFM
    e = trial.suggest_int("epochs", 3, 8)
    
    # Выбираем модель из данных
    choice_model = trial.suggest_categorical("model", ["ALS",  "LigthFM"])
    
    # Описываем гиперпараметры для каждой из выбранных моделей
    if choice_model == "ALS":
        model = ImplicitALSWrapperModel(
                        AlternatingLeastSquares(
                        factors=param["no_components"], 
                        random_state=RANDOM_STATE, 
                        num_threads=NUM_THREADS
                        ), 
                        fit_features_together=param["is_fitting_features"])
        model_name = f'ALS_n_f{param["no_components"]}_is_f{param["is_fitting_features"]}'      
    else:
        model = LightFMWrapperModel(
                LightFM(
                no_components=param["no_components"], 
                loss=param["loss"], 
                random_state=RANDOM_STATE,
                learning_rate=param["learning_rate"],
                user_alpha=param["user_alpha"],
                item_alpha=param["item_alpha"],

            ),
            epochs=e,
            num_threads=NUM_THREADS,
        )
        model_name = f'LightFM_loss{param["loss"]}_n_comp{param["no_components"]}_us_al{param["user_alpha"]}_it_al_{param["item_alpha"]}'
    
    print(f'Training model - {model}')
    #fit model
    model.fit(dataset)
    
    #predict_model
    recos = model.recommend(
        users=TEST_USERS,
        dataset=dataset,
        k=K_RECOS,
        filter_viewed=True,
    )
    
    #check metrics 
    metric_values = calc_metrics(metrics, recos, test, train)
    all_metrics[model_name] = metric_values
    map_10 = metric_values['MAP@10']
    
    print(f'Time : {time.time() - st_time}')
    return map_10

In [62]:
all_metrics = {}

# Базовая модель для популярного

model = PopularModel()
model_name = 'PopularModel'

model.fit(dataset)
    
#predict_model
recos = model.recommend(
    users=TEST_USERS,
    dataset=dataset,
    k=10,
    filter_viewed=True,
)

#check metrics 
metric_values = calc_metrics(metrics, recos, test, train)
all_metrics[model_name] = metric_values
print(metric_values['MAP@10'])

0.0742930700273141


In [63]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=50)

[I 2022-12-05 00:01:44,229] A new study created in memory with name: no-name-d0b033ba-9301-4fdb-a2e0-a1935fbec750


Training model - <rectools.models.lightfm.LightFMWrapperModel object at 0x7f5fd7094f90>


[I 2022-12-05 00:04:31,006] Trial 0 finished with value: 0.07569053377824472 and parameters: {'no_components': 20, 'loss': 'warp', 'learning_rate': 0.16152885610815282, 'item_alpha': 0.06392063237587971, 'user_alpha': 0.02556721488755738, 'is_fitting_features': True, 'epochs': 6, 'model': 'LigthFM'}. Best is trial 0 with value: 0.07569053377824472.


Time : 166.76368975639343
Training model - <rectools.models.implicit_als.ImplicitALSWrapperModel object at 0x7f5fd7008250>


[I 2022-12-05 00:11:40,176] Trial 1 finished with value: 0.07411843540881621 and parameters: {'no_components': 5, 'loss': 'warp', 'learning_rate': 0.06316801615158657, 'item_alpha': 0.021843420485024523, 'user_alpha': 0.18067083777128623, 'is_fitting_features': False, 'epochs': 8, 'model': 'ALS'}. Best is trial 0 with value: 0.07569053377824472.


...

Time : 201.66351795196533
Training model - <rectools.models.implicit_als.ImplicitALSWrapperModel object at 0x7f5f98daf090>


[I 2022-12-05 02:23:34,544] Trial 21 finished with value: 0.07363759258693267 and parameters: {'no_components': 8, 'loss': 'warp', 'learning_rate': 0.08117583496108888, 'item_alpha': 0.03137868877494353, 'user_alpha': 0.13282452629587183, 'is_fitting_features': False, 'epochs': 8, 'model': 'ALS'}. Best is trial 0 with value: 0.07569053377824472.


Time : 400.0953013896942
Training model - <rectools.models.implicit_als.ImplicitALSWrapperModel object at 0x7f5f98dadcd0>


[W 2022-12-05 02:29:15,350] Trial 22 failed because of the following error: KeyboardInterrupt()
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/site-packages/optuna/study/_optimize.py", line 196, in _run_trial
    value_or_values = func(trial)
  File "/tmp/ipykernel_28/1767026230.py", line 56, in objective
    filter_viewed=True,
  File "/opt/conda/lib/python3.7/site-packages/rectools/models/base.py", line 137, in recommend
    sorted_item_ids_to_recommend,
  File "/opt/conda/lib/python3.7/site-packages/rectools/models/vector.py", line 141, in _recommend_u2i
    scores = scores_calculator.calc(target_id)
  File "/opt/conda/lib/python3.7/site-packages/rectools/models/vector.py", line 98, in calc
    scores = self.objects_factors @ subject_factors
KeyboardInterrupt


KeyboardInterrupt: 

In [67]:
df_quality = pd.DataFrame(all_metrics)
df_quality.style.highlight_max(color='lightgreen', axis=1)

,PopularModel,LightFM_losswarp_n_comp20_us_al0.02556721488755738_it_al_0.06392063237587971,ALS_n_f5_is_fFalse,ALS_n_f28_is_fFalse,LightFM_losslogistic_n_comp29_us_al0.1802874629084228_it_al_0.13474229736050225,ALS_n_f21_is_fFalse,LightFM_losswarp_n_comp23_us_al0.08360038047566838_it_al_0.18230055510435225,ALS_n_f27_is_fFalse,ALS_n_f4_is_fFalse,ALS_n_f37_is_fTrue,ALS_n_f27_is_fTrue,LightFM_losswarp_n_comp11_us_al0.00032119254667496017_it_al_0.1214967993162966,LightFM_losswarp_n_comp12_us_al0.14330132372160465_it_al_0.0005964083627770719,LightFM_losswarp_n_comp14_us_al0.014675238544723856_it_al_0.040102931957721306,LightFM_losswarp_n_comp4_us_al0.13807778088116526_it_al_0.0004166410880281496,LightFM_losswarp_n_comp16_us_al0.03376672540431745_it_al_0.10118300459157903,LightFM_losswarp_n_comp17_us_al0.03136027594059083_it_al_0.11395448434505254,LightFM_losswarp_n_comp17_us_al0.04026893560063102_it_al_0.15315624834667882,LightFM_lossbpr_n_comp9_us_al0.07173369998854051_it_al_0.0993044859802907,LightFM_losswarp_n_comp17_us_al0.02992997141998289_it_al_0.08649714047619236,LightFM_losswarp_n_comp33_us_al0.04514449387733531_it_al_0.19920056336957917,LightFM_lossbpr_n_comp23_us_al0.004298897521840414_it_al_0.0610653097555256,ALS_n_f8_is_fFalse
Precision@5,0.051099,0.050244,0.048016,0.044883,0.000078,0.045114,0.046345,0.044727,0.046650,0.045996,0.046366,0.000018,0.000020,0.050140,0.000035,0.048705,0.047551,0.030897,0.000000,0.051082,0.050427,0.000000,0.047553
Recall@5,0.124408,0.122496,0.113872,0.105085,0.000121,0.105721,0.113998,0.104630,0.111176,0.110815,0.112088,0.000030,0.000022,0.122345,0.000036,0.118343,0.115323,0.072904,0.000000,0.124150,0.122686,0.000000,0.112387
Precision@6,0.046101,0.044972,0.043520,0.041771,0.000071,0.041916,0.042269,0.041530,0.042521,0.041227,0.041445,0.000024,0.000024,0.044738,0.000120,0.045006,0.045587,0.026096,0.000000,0.045342,0.044530,0.000000,0.043560
Recall@6,0.133311,0.130298,0.122537,0.116060,0.000126,0.116654,0.123615,0.115612,0.120492,0.118008,0.118883,0.000056,0.000039,0.129868,0.000220,0.130094,0.132032,0.073699,0.000000,0.131280,0.129030,0.000000,0.122391
Precision@7,0.041429,0.040130,0.039951,0.039045,0.000061,0.039088,0.038352,0.038926,0.039030,0.037269,0.037390,0.000028,0.000023,0.039768,0.000139,0.041672,0.040636,0.023046,0.000000,0.040258,0.039742,0.000000,0.040168
Recall@7,0.139172,0.134973,0.130065,0.125581,0.000126,0.125705,0.129757,0.125107,0.127964,0.123432,0.124068,0.000065,0.000041,0.134072,0.000285,0.139713,0.136606,0.075765,0.000000,0.135312,0.133364,0.000000,0.130448
Precision@8,0.038100,0.036600,0.037019,0.036725,0.000053,0.036715,0.034431,0.036658,0.036074,0.034041,0.034194,0.000051,0.000020,0.036125,0.000128,0.038367,0.036685,0.021063,0.000000,0.036421,0.035746,0.000000,0.037317
Recall@8,0.145802,0.139904,0.136585,0.133741,0.000126,0.134110,0.132604,0.133490,0.134025,0.127974,0.128833,0.000192,0.000041,0.138473,0.000290,0.146810,0.140313,0.078725,0.000000,0.139321,0.136621,0.000000,0.137508
Precision@9,0.035455,0.033930,0.034640,0.034723,0.000111,0.034693,0.031340,0.034747,0.033671,0.031498,0.031556,0.000076,0.000020,0.033203,0.000143,0.035980,0.033417,0.019660,0.000000,0.033452,0.032603,0.000000,0.035025
Recall@9,0.152458,0.145216,0.142556,0.141241,0.000312,0.141599,0.135208,0.141324,0.139671,0.132455,0.132827,0.000329,0.000043,0.142752,0.000361,0.154826,0.143672,0.082282,0.000000,0.143219,0.139718,0.000000,0.143945


In [68]:
optuna.visualization.plot_optimization_history(study)

In [69]:
best_params = study.best_params

In [70]:
best_params

{'no_components': 20,
 'loss': 'warp',
 'learning_rate': 0.16152885610815282,
 'item_alpha': 0.06392063237587971,
 'user_alpha': 0.02556721488755738,
 'is_fitting_features': True,
 'epochs': 6,
 'model': 'LigthFM'}

### Cохраним наш блокнот, и обучим модель с наилучшими параметрами на всех данных 